#**03 - modelo con función de densidad gaussiana**

**Miembros del grupo:**

* Cristian Camilo Serna Betancur, CC 1018351871, Ingeniería de Sistemas
* Diego Alonso Herrera Ramírez, CC 70908268, Ingeniería de Sistemas
* Sharid Samantha Madrid Ospina, CC 1001652997, Ingeniería de Sistemas


**Base de datos:**
* [hotel_booking](https://www.kaggle.com/datasets/saadharoon27/hotel-booking-dataset/data): Contiene información sobre reservas de hotel, que abarca detalles sobre los huéspedes, sus reservas y atributos del hote

In [ ]:
import pandas as pd

# Se carga el conjunto de datos original
db = pd.read_csv("https://raw.githubusercontent.com/Udeaproject/IA-system/main/preprocessed_hotel_booking2.csv", delimiter=",")

In [ ]:
from sklearn.model_selection import train_test_split

train_db, test_db = train_test_split(db, test_size=0.2, random_state=40)
x_test = test_db.drop(['is_canceled'], axis=1).copy()
y_test = test_db['is_canceled']

In [ ]:
import numpy as np

# Entrenamiento
canceled_1 = train_db[train_db["is_canceled"] == 1]
canceled_1 = canceled_1.drop(["is_canceled"], axis=1).copy()
canceled_miu_1 = np.array([np.mean(canceled_1[column]) for column in canceled_1.columns])
canceled_cov_1 = np.cov(canceled_1.to_numpy(), rowvar=False)
canceled_det_1 = np.linalg.det(canceled_cov_1)
canceled_inv_1 = np.linalg.inv(canceled_cov_1)

canceled_0 = train_db[train_db["is_canceled"] == 0]
canceled_0 = canceled_0.drop(["is_canceled"], axis=1).copy()
canceled_miu_0 = np.array([np.mean(canceled_0[column]) for column in canceled_0.columns])
canceled_cov_0 = np.cov(canceled_0.to_numpy(), rowvar=False)
# Se aplica regularizacion debido a que el determinante estaba siendo igual a 0
canceled_cov_0 = canceled_cov_0 + np.eye(canceled_cov_0.shape[0]) * 1e-6
canceled_det_0 = np.linalg.det(canceled_cov_0)
canceled_inv_0 = np.linalg.inv(canceled_cov_0)

In [ ]:
# Modelo
def fdg_classifier(m):
    t1 = 1 / ((pow((2 * np.pi), (np.size(db, 1) / 2))) * np.sqrt(canceled_det_0))
    prod1 = np.dot((m - canceled_miu_0).reshape(31, 1).T, canceled_inv_0)
    prod2 = np.dot(prod1, (m - canceled_miu_0).reshape(31, 1))[0, 0]
    t2 = np.exp(-0.5 * (prod2))
    p0 = t1 * t2

    t1 = 1 / ((pow((2 * np.pi), (np.size(db, 1) / 2))) * np.sqrt(canceled_det_1))
    prod1 = np.dot((m - canceled_miu_1).reshape(31, 1).T, canceled_inv_1)
    prod2 = np.dot(prod1, (m - canceled_miu_1).reshape(31, 1))[0, 0]
    t2 = np.exp(-0.5 * (prod2))
    p1 = t1 * t2

    if p1 > p0:
        return 1
    else:
        return 0

In [ ]:
def fdg_predict(x_test):
    return x_test.apply(lambda row: fdg_classifier(row.values.reshape(1, -1)[0]), axis=1)

def error(y_pred, y_test):
    return np.mean(y_pred != y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

y_pred = fdg_predict(x_test)
print("La eficiencia del modelo es: ", np.round((1 - error(y_pred, y_test)) * 100, 2), "%")
print("Matriz de foncusion: ", confusion_matrix(y_test, y_pred))
print("F1-score: ", f1_score(y_test, y_pred))

La eficiencia del modelo es:  99.34 %
Matriz de foncusion:  [[7367   99]
 [   2 7943]]
F1-score:  0.9936823669231251
